In [3]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
import pandas as pd


In [4]:
# 从附件1中读取数据。
df_shipment = pd.read_excel("./附件表/附件1-商家历史出货量表.xlsx")
df_shipment['合并主键']=df_shipment['seller_no']+df_shipment['product_no']+df_shipment['warehouse_no']


In [5]:
data1=pd.read_excel('./结果表/结果表1-预测结果表.xlsx')
data1['合并主键']=data1['seller_no']+data1['product_no']+data1['warehouse_no']

In [6]:
df6=pd.read_excel('./附件表/附件6-促销期间商家出货量表.xlsx')
df6['合并主键']=df6['seller_no']+df6['product_no']+df6['warehouse_no']

In [7]:
df1=df_shipment.groupby('合并主键').mean()['qty']
df1=df1.reset_index()

In [8]:
df6=df6.groupby('合并主键').mean()
df6=df6.reset_index()

In [12]:
import numpy as np

def weighted_mean_absolute_percentage_error(y_true, y_pred):
    """
    计算1-WMAPE（Weighted Mean Absolute Percentage Error）。

    参数：
    y_true: 真实需求量的numpy数组或列表
    y_pred: 预测需求量的numpy数组或列表

    返回值：
    1-WMAPE
    """
    # 将输入转换为numpy数组
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # 计算APE
    ape = np.abs(y_true - y_pred)
    
    # 计算1-WMAPE
    wmape = 1 - np.sum(ape) / np.sum(y_true)
    
    return wmape

def getdata(x):
    try:
        return round(x[0][0])
    except:
        try:
            return round(x[0])
        except:
            return round(x[0][0][0])

In [11]:
dd=df_shipment['合并主键'].unique()

In [23]:
%%time
data = []

total_iterations = len(dd)
output_frequency = total_iterations // 100  # 每隔1%输出一次

for index, d in enumerate(dd):
    # 计算已完成的百分比
    progress_percentage = (index + 1) / total_iterations * 100

    # 如果达到输出百分比的条件，就输出一次
    if index % output_frequency == 0:
        print(f"Progress: {progress_percentage:.1f}%")

        
    try:
        temp=df_shipment[df_shipment['合并主键']==d]
        b=df6[df6['合并主键']==d]['qty'].sum()/df1[df1['合并主键']==d]['qty'].sum()
        if float(b)==float("inf"):
            b=1+df6[df6['合并主键']==d]['qty'].sum()
            
        seller_no=temp['seller_no'].unique()[0]
        product_no=temp['product_no'].unique()[0]
        warehouse_no=temp['warehouse_no'].unique()[0]
        # 根据日期排序
        temp['date'] = pd.to_datetime(temp['date'])
        temp = temp.sort_values(by='date')
        temp_=temp.iloc[-5:]
        tempp_=temp.iloc[:-5]
                # 将数据按15天为一个周期进行合并，首先需要计算最近的日期与2023-05-15的差值，然后从这个差值开始合并
        days_diff = (tempp_["date"].max() - tempp_.iloc[-1,3]).days

        # 使用rolling方法合并数据，窗口大小为20
        tempp_['Cumulative_Sales'] = tempp_['qty'].rolling(window=20, min_periods=1).sum()

        # 根据差值筛选出合并后的数据
        df_grouped = tempp_.iloc[days_diff::20].reset_index(drop=True)

        # 生成对应的时间周期
        df_grouped["Start_Date"] = df_grouped["date"] - pd.Timedelta(days=20)
        df_grouped["End_Date"] = df_grouped["date"]
        dftemp=df_grouped[['Start_Date', 'End_Date', 'Cumulative_Sales']].iloc[1:]
        latsvlaue=temp_['qty'].sum()+data1[data1['合并主键']=='seller_16product_1187wh_33']['forecast_qty'].sum()
        # 新行的数据
        new_row = {'Start_Date': '2023-05-11  00:00:00', 'End_Date': '2023-05-30  00:00:00','Cumulative_Sales':latsvlaue}

        # 使用append方法将新行添加到DataFrame的末尾
        dftemp = dftemp.append(new_row, ignore_index=True)
        dftemp['y']=dftemp['Cumulative_Sales'].shift(-1)
        dftemp=dftemp.dropna()
        X_train, X_test, y_train, y_test = train_test_split(dftemp['Cumulative_Sales'].values.reshape(-1, 1), dftemp['y'].values.reshape(-1, 1), test_size=0.1,shuffle=False )
        models = {
        "Linear Regression": LinearRegression(),
        "Random Forest": RandomForestRegressor(),
        "Decision Tree": DecisionTreeRegressor(),
        "MLP": MLPRegressor(),
        "XGBoost": xgb.XGBRegressor(objective='reg:squarederror')
    }

        best_model = None
        best_error = -1  # 最大值初始化为负数

        for model_name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            error  = weighted_mean_absolute_percentage_error(y_test, y_pred)


            if error > best_error:
                best_error = error
                best_model = model

            # 预测未来一个时间周期的销售量         	seller_no	product_no	warehouse_no
        try:
            future_sales_pre = getdata(best_model.predict(y_test))
            future_sales = round(future_sales_pre*b)
            data.append([seller_no,product_no,warehouse_no,future_sales_pre,future_sales,best_model,best_error])
        except:
            future_sales_pre = getdata(best_model.predict(y_test))
            future_sales = round(future_sales_pre*b)
            data.append([seller_no,product_no,warehouse_no,future_sales_pre,future_sales,best_model,best_error])
    except:
        data.append([seller_no,product_no,warehouse_no,getdata(y_test),round(getdata(y_test)*b),0,0])


Progress: 0.1%
Progress: 1.0%
Progress: 2.0%
Progress: 2.9%
Progress: 3.9%
Progress: 4.8%
Progress: 5.8%
Progress: 6.7%
Progress: 7.7%
Progress: 8.6%
Progress: 9.6%
Progress: 10.5%
Progress: 11.5%
Progress: 12.4%
Progress: 13.4%
Progress: 14.3%
Progress: 15.3%
Progress: 16.2%
Progress: 17.2%
Progress: 18.1%
Progress: 19.1%
Progress: 20.0%
Progress: 21.0%
Progress: 21.9%
Progress: 22.9%
Progress: 23.8%
Progress: 24.8%
Progress: 25.8%
Progress: 26.7%
Progress: 27.7%
Progress: 28.6%
Progress: 29.6%
Progress: 30.5%
Progress: 31.5%
Progress: 32.4%
Progress: 33.4%
Progress: 34.3%
Progress: 35.3%
Progress: 36.2%
Progress: 37.2%
Progress: 38.1%
Progress: 39.1%
Progress: 40.0%
Progress: 41.0%
Progress: 41.9%
Progress: 42.9%
Progress: 43.8%
Progress: 44.8%
Progress: 45.7%
Progress: 46.7%
Progress: 47.6%
Progress: 48.6%
Progress: 49.5%
Progress: 50.5%
Progress: 51.5%
Progress: 52.4%
Progress: 53.4%
Progress: 54.3%
Progress: 55.3%
Progress: 56.2%
Progress: 57.2%
Progress: 58.1%
Progress: 59.1%
Pro

In [24]:
dfoutput=pd.DataFrame(data)
dfoutput.columns=['seller_no','product_no','warehouse_no','future_sales_pre','future_sales','best_model','best_error']
dfoutput

,seller_no,product_no,warehouse_no,future_sales_pre,future_sales,best_model,best_error
0,seller_19,product_448,wh_30,101,140,MLPRegressor(),-0.628428
1,seller_11,product_148,wh_1,138,238,MLPRegressor(),-0.739707
2,seller_11,product_132,wh_16,44,48,"(DecisionTreeRegressor(max_features='auto', ra...",0.750000
3,seller_11,product_170,wh_9,66,111,"XGBRegressor(base_score=0.5, booster='gbtree',...",0.984133
4,seller_11,product_184,wh_18,126,94,LinearRegression(),0.860121
...,...,...,...,...,...,...,...
1991,seller_32,product_1061,wh_1,49,216,DecisionTreeRegressor(),-0.886364
1992,seller_32,product_1112,wh_29,188,1466,LinearRegression(),-0.641973
1993,seller_15,product_830,wh_1,547,357,DecisionTreeRegressor(),0.140426
1994,seller_21,product_494,wh_33,47,0,LinearRegression(),0.527453


In [25]:
dfoutput[dfoutput['best_model']==0]

,seller_no,product_no,warehouse_no,future_sales_pre,future_sales,best_model,best_error
8,seller_28,product_807,wh_43,89,123,0,0.0
49,seller_11,product_178,wh_9,459,360,0,0.0
50,seller_25,product_1405,wh_1,468,1088,0,0.0
54,seller_15,product_856,wh_1,67,196,0,0.0
59,seller_33,product_1345,wh_1,815,5974,0,0.0
...,...,...,...,...,...,...,...
1912,seller_25,product_1391,wh_1,128,339,0,0.0
1919,seller_24,product_946,wh_1,132,841,0,0.0
1947,seller_15,product_897,wh_48,109,59,0,0.0
1971,seller_25,product_1412,wh_32,377,426,0,0.0


In [26]:
dfoutput[dfoutput['best_model']!=0]['best_error'].mean()

0.5160705821100041

In [28]:
data3=pd.read_excel('./结果表/结果表3-预测结果表.xlsx')
data3

,seller_no,product_no,warehouse_no,date,forecast_qty


In [29]:
data3['seller_no']=dfoutput['seller_no']
data3['product_no']=dfoutput['product_no']
data3['warehouse_no']=dfoutput['warehouse_no']
data3['date']='2023-06-01 至2023-06-20'
data3['forecast_qty']=dfoutput['future_sales']

In [30]:
data3.to_excel('./结果表/结果表3-预测结果表.xlsx',index=None)